# dydx public API

This notebook shows how to use dydx API to acquire historic and live data. Python package dydx-v3-python and web3 are needed.

Refer to dydx docs for details: https://web3py.readthedocs.io/en/stable/quickstart.html

In [165]:
## Required packages 
## pip install dydx-v3-python
## pip install web3

###################
## Load packages ##
###################

from dydx3 import Client
from web3 import Web3
from dydx3.helpers.request_helpers import generate_query_path
from dydx3.helpers.requests import request
import requests
import json
import pandas as pd
import numpy as np


In [169]:
class Public(object):

    def __init__(
        self,
        host,
    ):
        self.host = host

    ## Request Helpers

    def _get(self, request_path, params={}):
        return request(
            generate_query_path(self.host + request_path, params),
            'get',
        )

    def _put(self, endpoint, data):
        return request(
            self.host + '/v3/' + endpoint,
            'put',
            {},
            data,
        )

    ## Requests Data
    def get_market(self):
        uri = '/'.join(['/v3/markets'])
        return self._get(uri).data 
    
    def get_orderbook(self, market):
        uri = '/'.join(['/v3/orderbook', market])
        return self._get(uri).data 

     
    def get_trade(self, market, startingBeforeOrAt=None, limit=None):
        uri = '/'.join(['/v3/trades', market])
        if(startingBeforeOrAt is None or limit is None):
            uri_full = uri
        else:
            uri_full = uri + '?startingBeforeOrAt=' + startingBeforeOrAt + '&limit=' + limit
        return self._get(uri_full).data 
   

    def get_stats(self, market, days=1):
        uri = '/'.join(['/v3/stats', market])
        return self._get(uri,{'days': days},).data


    def get_historical_funding(self, market, effective_before_or_at=None):
        uri = '/'.join(['/v3/historical-funding', market])
        return self._get(uri,{'effectiveBeforeOrAt': effective_before_or_at},).data 
    
    def get_config(self):
        uri = '/'.join(['/v3/config'])
        return self._get(uri).data 

    def get_leaderboard(self, period="DAILY", startingBeforeOrAt=None, sortBy='PERCENT', limit='10'):
        uri = '/'.join(['/v3/leaderboard-pnl'])
        return self._get(uri,{'period': period,'startingBeforeOrAt': startingBeforeOrAt, 'sortBy': sortBy, 'limit': limit},).data 
   
    def get_current_hedgies(self):
        uri = '/'.join(['/v3/hedgies/current'])
        return self._get(uri).data 
    
    def get_hist_hedgies(self, nftRevealType="Day", start=None, end=None):
        uri = '/'.join(['/v3/hedgies/history'])
        return self._get(uri,{'nftRevealType': nftRevealType,'start': start, 'end': end},).data 


In [170]:
## Public api client
client = Public(host="https://api.dydx.exchange")

In [113]:
#################
## Get Markets ##
#################
## Get all markets
market = pd.DataFrame(client.get_market()['markets'])
market

,BTC-USD,SUSHI-USD,AVAX-USD,1INCH-USD,ETH-USD,XMR-USD,COMP-USD,ALGO-USD,BCH-USD,CRV-USD,...,UMA-USD,AAVE-USD,ADA-USD,SNX-USD,FIL-USD,ZEC-USD,YFI-USD,LINK-USD,DOT-USD,MATIC-USD
market,BTC-USD,SUSHI-USD,AVAX-USD,1INCH-USD,ETH-USD,XMR-USD,COMP-USD,ALGO-USD,BCH-USD,CRV-USD,...,UMA-USD,AAVE-USD,ADA-USD,SNX-USD,FIL-USD,ZEC-USD,YFI-USD,LINK-USD,DOT-USD,MATIC-USD
status,ONLINE,ONLINE,ONLINE,ONLINE,ONLINE,ONLINE,ONLINE,ONLINE,ONLINE,ONLINE,...,ONLINE,ONLINE,ONLINE,ONLINE,ONLINE,ONLINE,ONLINE,ONLINE,ONLINE,ONLINE
baseAsset,BTC,SUSHI,AVAX,1INCH,ETH,XMR,COMP,ALGO,BCH,CRV,...,UMA,AAVE,ADA,SNX,FIL,ZEC,YFI,LINK,DOT,MATIC
quoteAsset,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
stepSize,0.0001,0.1,0.1,1,0.001,0.01,0.01,1,0.01,1,...,0.1,0.01,1,0.1,0.1,0.01,0.0001,0.1,0.1,1
tickSize,1,0.001,0.01,0.001,0.1,0.1,0.1,0.001,0.1,0.001,...,0.01,0.01,0.001,0.01,0.01,0.1,1,0.001,0.01,0.001
indexPrice,39432.9695,3.2946,84.4375,1.4488,2819.2200,158.8194,114.6448,0.8805,315.6900,2.4220,...,4.9863,138.5800,0.9013,3.9420,19.3501,108.9022,21268.0000,14.7992,18.2921,1.5427
oraclePrice,39419.5900,3.2920,84.3900,1.4480,2816.6819,158.8202,114.3900,0.8812,315.7880,2.4200,...,4.9930,138.7900,0.9004,3.9410,19.3418,108.9000,21221.6700,14.7700,18.2775,1.5419
priceChange24H,494.749500,0.023595,6.487488,0.034964,118.363700,7.209370,2.529954,0.071872,11.687390,0.075034,...,0.096335,-1.522820,0.020831,-0.087702,0.610060,1.372218,1369.470000,0.705146,1.762133,0.025679
nextFundingRate,-0.0000218799,0.0000011633,0.0000094390,-0.0000255221,-0.0000030657,-0.0000730795,0.0000125000,-0.0000020311,0.0000027774,0.0000125000,...,0.0000125000,0.0000064419,-0.0000195738,-0.0000595796,0.0000099778,0.0000125000,0.0000108326,-0.0000051743,-0.0000306293,0.0000125000


In [114]:
####################
## Get Order Book ##
####################
## Returns the active orderbook for a market. All bids and asks that are fillable are returned.
bid = pd.DataFrame(client.get_orderbook('BTC-USD')['bids'])
display(bid)
ask = pd.DataFrame(client.get_orderbook('BTC-USD')['asks'])
display(ask)

,size,price
0,2.0362,39381
1,0.0402,39379
2,3.5965,39378
3,1.058,39376
4,0.0448,39375
...,...,...
286,0.042,9321
287,0.2778,3600
288,0.2,2006
289,5,1575


,size,price
0,1.0581,39382
1,0.4095,39387
2,1.3281,39388
3,1.5581,39389
4,0.7525,39390
...,...,...
227,0.002,80000
228,128,87351
229,0.005,88000
230,0.0288,200000


In [99]:
################
## Get Trades ##
################

## Get trades from a market
trade = client.get_trade('BTC-USD')
trade = pd.DataFrame(trade['trades'])
display(trade)

## Get trades from a market with defined time & limit
s_trade = client.get_trade('BTC-USD','2021-09-05T17:33:43.163Z','1')
s_trade =  pd.DataFrame(s_trade['trades'])
display(s_trade)


,side,size,price,createdAt
0,SELL,0.001,39368,2022-02-26T21:47:46.987Z
1,SELL,0.153,39367,2022-02-26T21:47:44.377Z
2,SELL,0.2593,39377,2022-02-26T21:47:44.102Z
3,SELL,0.3248,39377,2022-02-26T21:47:44.075Z
4,SELL,0.153,39377,2022-02-26T21:47:44.066Z
...,...,...,...,...
95,SELL,0.071,39434,2022-02-26T21:45:00.076Z
96,SELL,0.007,39434,2022-02-26T21:45:00.046Z
97,SELL,0.043,39434,2022-02-26T21:45:00.039Z
98,SELL,0.007,39434,2022-02-26T21:44:59.990Z


,side,size,price,createdAt
0,BUY,0.001,50377,2021-09-05T17:20:08.497Z


In [133]:
######################
## Get Market Stats ##
######################
## Get an individual market's statistics over a set period of time or all available periods of time.
## Specified day range for the statistics to have been compiled over. Can be one of 1, 7, 30. Defaults to 1.
stats = client.get_stats("BTC-USD", days=1)
stats = pd.DataFrame(stats['markets'])
stats

,BTC-USD
baseVolume,11142.3668
close,39013
fees,97441.172919
high,40400
low,38590
market,BTC-USD
open,39357
quoteVolume,438039938.7501
type,PERPETUAL


In [134]:
############################
## Get Historical Funding ##
############################
## Get the historical funding rates for a market.
funding = client.get_historical_funding("BTC-USD")
funding = pd.DataFrame(funding['historicalFunding'])
funding

,market,rate,price,effectiveAt
0,BTC-USD,-0.0000148635,39147.3810700700,2022-02-26T23:00:00.000Z
1,BTC-USD,0.0000008082,39459.4829995185,2022-02-26T22:00:00.000Z
2,BTC-USD,-0.0000004312,39394.4999994710,2022-02-26T21:00:00.000Z
3,BTC-USD,-0.0000109556,39295.0899992138,2022-02-26T20:00:00.000Z
4,BTC-USD,-0.0000074303,39085.2189995348,2022-02-26T19:00:00.000Z
...,...,...,...,...
95,BTC-USD,-0.0000140274,38250.9900000878,2022-02-23T00:00:00.000Z
96,BTC-USD,-0.0000128150,37883.9999996126,2022-02-22T23:00:00.000Z
97,BTC-USD,-0.0000148562,37862.7500007860,2022-02-22T22:00:00.000Z
98,BTC-USD,0.0000004543,37945.3970002942,2022-02-22T21:00:00.000Z


In [141]:
#######################
## Get Configuration ##
#######################
## Get any global configuration variables for the exchange as a whole.
config = pd.DataFrame(client.get_config())
config

,collateralAssetId,collateralTokenAddress,defaultMakerFee,defaultTakerFee,exchangeAddress,maxExpectedBatchLengthMinutes,maxFastWithdrawalAmount,cancelOrderRateLimiting,placeOrderRateLimiting
maxPointsMulti,0x02893294412a4c8f915f75892b395ebbf6859ec246ec...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0.0002,0.0005,0xD54f502e184B6B739d7D27a6410a67dc462D69c8,720,200000,3.0,NaN
maxPointsSingle,0x02893294412a4c8f915f75892b395ebbf6859ec246ec...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0.0002,0.0005,0xD54f502e184B6B739d7D27a6410a67dc462D69c8,720,200000,8500.0,NaN
windowSecMulti,0x02893294412a4c8f915f75892b395ebbf6859ec246ec...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0.0002,0.0005,0xD54f502e184B6B739d7D27a6410a67dc462D69c8,720,200000,10.0,NaN
windowSecSingle,0x02893294412a4c8f915f75892b395ebbf6859ec246ec...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0.0002,0.0005,0xD54f502e184B6B739d7D27a6410a67dc462D69c8,720,200000,10.0,NaN
maxPoints,0x02893294412a4c8f915f75892b395ebbf6859ec246ec...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0.0002,0.0005,0xD54f502e184B6B739d7D27a6410a67dc462D69c8,720,200000,NaN,1000.0
windowSec,0x02893294412a4c8f915f75892b395ebbf6859ec246ec...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0.0002,0.0005,0xD54f502e184B6B739d7D27a6410a67dc462D69c8,720,200000,NaN,10.0
targetNotional,0x02893294412a4c8f915f75892b395ebbf6859ec246ec...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0.0002,0.0005,0xD54f502e184B6B739d7D27a6410a67dc462D69c8,720,200000,NaN,40000.0
minLimitConsumption,0x02893294412a4c8f915f75892b395ebbf6859ec246ec...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0.0002,0.0005,0xD54f502e184B6B739d7D27a6410a67dc462D69c8,720,200000,NaN,4.0
minMarketConsumption,0x02893294412a4c8f915f75892b395ebbf6859ec246ec...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0.0002,0.0005,0xD54f502e184B6B739d7D27a6410a67dc462D69c8,720,200000,NaN,20.0
minTriggerableConsumption,0x02893294412a4c8f915f75892b395ebbf6859ec246ec...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0.0002,0.0005,0xD54f502e184B6B739d7D27a6410a67dc462D69c8,720,200000,NaN,100.0


In [159]:
#####################
## Get Leaderboard ##
#####################
## Get the top PNLs for a specified period and how they rank against each other.

leader = client.get_leaderboard(period="DAILY", startingBeforeOrAt=None, sortBy='PERCENT', limit='10')
leader = pd.DataFrame(leader['topPnls'])
leader

,username,ethereumAddress,absolutePnl,percentPnl,absoluteRank,percentRank,seasonExpectedOutcome,hedgieWon,prizeWon
0,None,None,21436.488035,1.163536,39,1,None,None,None
1,None,0xce20ab2854b22b153c72531bcfe010aa682ca924,3508.097512,0.640805,96,2,None,None,None
2,None,0xe93167c44dbce5aa2f3cc6256dd9e027acfa8899,283.071847,0.566144,243,3,None,None,None
3,None,0x463ab6900a32e6c7881ac268b6be5bad7be6ed0e,22990.374257,0.449875,37,4,None,None,None
4,None,0x5fc767917545a37b36844b5d525bb2734eab6501,171.974359,0.343949,281,5,None,None,None
5,None,0x796c56deae095f927f4cd0680091a05ed3a2627c,1923.422242,0.327399,123,6,None,None,None
6,None,0xcf711f7eb678bc36e6f72d9e003ab23b46212f7d,508.693849,0.321658,195,7,None,None,None
7,None,0x14011ecb81c4afc94e8fd89dd7c00384ac49a95d,2072.095476,0.319493,117,8,None,None,None
8,None,0xe629e1f7d250d39af4d704b486b094a4ba91ef3b,1852.288176,0.303993,126,9,None,None,None
9,None,0xd3423983bbd7ac9d360fb63dfc1b86d37f4d46bf,290.085682,0.290846,241,10,None,None,None


In [168]:
#########################
## Get Current Hedgies ##
#########################
## Get the currently revealed Hedgies for competition distribution.

hedgies = client.get_current_hedgies()
h_daily = pd.DataFrame(hedgies['daily'])
h_weekly = pd.DataFrame(hedgies['weekly'])
display(h_daily)
display(h_weekly)

,blockNumber,tokenIds,competitionPeriod
0,14288867,3554,21


,blockNumber,tokenIds,competitionPeriod
0,14256533,4193,3
1,14256533,3321,3
2,14256533,4030,3
3,14256533,2974,3
4,14256533,3614,3
5,14256533,2828,3
6,14256533,3572,3
7,14256533,4138,3
8,14256533,2543,3
9,14256533,3832,3


In [171]:
#########################
## Get History Hedgies ##
#########################
## Get the historically revealed Hedgies from competition distributions.
hist_hedgies_d = client.get_hist_hedgies(nftRevealType="daily")
hh_daily = pd.DataFrame(hist_hedgies_d['historicalTokenIds'])
display(hh_daily)

hist_hedgies_w = client.get_hist_hedgies(nftRevealType="weekly")
hh_weekly = pd.DataFrame(hist_hedgies_w['historicalTokenIds'])
display(hist_hedgies_w)

,blockNumber,tokenIds,competitionPeriod
0,14282380,[3420],20
1,14275948,[3219],19
2,14269450,[2755],18
3,14263023,[3901],17
4,14256533,[2920],16
5,14250118,[2640],15
6,14243575,[3517],14
7,14237138,[3264],13
8,14230777,[4188],12
9,14224315,[3001],11


{'historicalTokenIds': [{'blockNumber': '14211356',
   'tokenIds': ['3962', '4139', '3115', '2623', '2570', '2598'],
   'competitionPeriod': 2},
  {'blockNumber': '14166105',
   'tokenIds': ['2911', '3054', '2698'],
   'competitionPeriod': 1}]}